<img title="GitHub Octocat" src='./img/Octocat.jpg' style='height: 60px; padding-right: 15px' alt="Octocat" align="left"> This notebook is part of a GitHub repository: https://github.com/pessini/moby-bikes 
<br>MIT Licensed
<br>Author: Leandro Pessini

<img title="Moby Bikes" src="https://i.ytimg.com/vi/-s8er6tHD3o/maxresdefault.jpg" style="height:400px; border-radius: 10px" alt="Moby Bikes" class="img-fluid">

<div class="list-group" id="list-tab" role="tablist">
  <h1 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">&nbsp;Table of Contents:</h1>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab" aria-controls="profile">1. Introduction<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages">2. Feature Engineering<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3" role="tab" aria-controls="messages">3. EDA (Exploratory Data Analysis)<span class="badge badge-primary badge-pill">3</span></a>
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#4" role="tab" aria-controls="messages">4. Model<span class="badge badge-primary badge-pill">4</span></a>
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#5" role="tab" aria-controls="messages">5. Evaluation<span class="badge badge-primary badge-pill">5</span></a>    
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#6" role="tab" aria-controls="messages">6. Conclusion<span class="badge badge-primary badge-pill">6</span></a>
</div>

---

<a id="1"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">1- Introduction</p>

## Datasets

Dataset provided by [Moby Bikes](https://data.gov.ie/dataset/moby-bikes) through a public [API](https://data.smartdublin.ie/mobybikes-api). 

Dataset provided by [Met Éireann](https://www.met.ie/) through a public [API](https://data.gov.ie/organization/meteireann). 


The **aim** of this report is to map how this disease spread throughout the island and provide insights on possible spots and species that might need extra attention from the scientists who investigate this constant threat to resident birds.

## Questions

- What species have shown to be the most affected with Bird Flu?
- What are the most frequent locations where captured birds have been detected with Avian Flu?
- November is the month with the highest presence as mentioned? What are the months with the highest proportion of infected birds?
- The percentage of infected birds have increased during the years?
- What is the proportion of birds targeted with Avian Flu on each Council / County?
- Which areas present statistically significant incidence of Bird Flu?


>https://mobidev.biz/blog/machine-learning-methods-demand-forecasting-retail

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
from conn import mongodb
import importlib
from pymongo import MongoClient
from urllib.parse import quote_plus
importlib.reload(mongodb)

<module 'conn.mongodb' from '/Users/pessini/Dropbox/Data-Science/moby-bikes/notebooks/conn/mongodb.py'>

In [3]:
def _connect_mongo(host, port, username, password, db_name):
    """ A util for making a connection to mongo """

    if username and password:
        try:
            mongo_uri = f'mongodb://{username}:{quote_plus(password)}@{host}:{port}/{db_name}'
            conn = MongoClient(mongo_uri)
        except:
            print('Could not connect to MongoDB')
    else:
        conn = MongoClient(host, port)

    return conn

In [4]:
def read_mongo(query={}, collection='', no_id=True):
    """ Read from Mongo and Store into a DataFrame """
    df = None
    try:
        # Connect to MongoDB
        conn = _connect_mongo(host=mongodb.host, port=mongodb.port, username=mongodb.user_name, password=mongodb.pass_word, db_name=mongodb.db_name)
        db = conn.mobybikes # switch to the database

        if collection in db.list_collection_names():
            
            # Make a query to the specific DB and Collection and store into a Dataframe
            data = db[collection].find(query)
            df =  pd.DataFrame(list(data))
            
            # Delete the _id
            if no_id:
                del df['_id']
        else:
            print(f'Collection {collection} was not found!')
            pass

        # close mongodb connection
        conn.close()
    except:
        print('Could not query MongoDB')
    
    return df


In [5]:
# from bson.objectid import ObjectId

# query = {
#     '_id': ObjectId('')
# }

historical_data = read_mongo(collection='historical')

In [ ]:
historical_data.columns = historical_data.columns.str.lower()
historical_data.head()

In [ ]:
historical_data.isnull().sum()

## Weather Data - Met Éireann

About the weather data there are two important decisions. One is about from **which station** the **historical data will be collected** and the other one is about the **frequency of data**, which can be **hourly or daily**.

### Station Name: **PHOENIX PARK**

In [ ]:
phoenixpark_weather_hourly = pd.read_csv('../data/raw/hly175.csv')
phoenixpark_weather_hourly.head()

In [ ]:
phoenixpark_weather_daily = pd.read_csv('../data/raw/dly175.csv')
phoenixpark_weather_daily.head()

### Station Name: **DUBLIN AIRPORT**

In [ ]:
dublin_airport_weather_hourly = pd.read_csv('../data/raw/hly532.csv')
dublin_airport_weather_hourly.head()

## Phoenix Park Station vs Dublin Aiport Station
Geographically, the station at Phoenix Park would be the most suitable choice but unfortunately, they do not collect Wind information which in Ireland plays an important role when deciding to go cycling or not. For those who are not familiar with Irish weather, it rains a lot and mostly we do not have much choice about it but the wind is something that can prevent you go outside or choose a different kind of transport. Heavy rain is kind of rare on the other hand.

## Hourly vs Daily data
On this subject, daily data for the business would make more sense but because the weather is so unpredictable in Ireland (it can completely change in an hour), the best option would be hourly especially if we are looking at a historical perspective. For simplicity and better planning, we can always aggregate the predicted results by day.

In [ ]:
dublin_airport_weather_hourly['date'] = pd.to_datetime(dublin_airport_weather_hourly['date'])

In [ ]:
dublin_airport_weather_hourly.info()

In [ ]:
dublin_airport_weather_hourly.head()

In [ ]:
recent_dubairport_data = dublin_airport_weather_hourly.copy()

start_date_hist = datetime(2020, 9, 23) # first day registered
end_date_hist = datetime(2021, 8, 31) # last day used as historical data

recent_dubairport_data = recent_dubairport_data[(recent_dubairport_data.date >= start_date_hist) & (recent_dubairport_data.date <= end_date_hist)]

In [ ]:
len(dublin_airport_weather_hourly), len(recent_dubairport_data)

In [ ]:
columns_to_drop = ['ind','ind.1','ind.2','ind.3','vappr','msl','ind.4','wddir','ww','w','sun','vis','clht','clamt']
weather_data = recent_dubairport_data.drop(columns=columns_to_drop)
weather_data.to_csv('../data/interim/hist_weather_data.csv')

In [ ]:
weather_data.head()

## Hypothesis

Hourly trend: There must be high demand during office timings. Early morning and late evening can have different trend (cyclist) and low demand during 10:00 pm to 4:00 am.

Daily Trend: Registered users demand more bike on weekdays as compared to weekend or holiday.

Rain: The demand of bikes will be lower on a rainy day as compared to a sunny day. Similarly, higher humidity will cause to lower the demand and vice versa.

Temperature: In Ireland, temperature has positive correlation with bike demand.

Time: Total demand should have higher contribution of registered user as compared to casual because registered user base would increase over time.

Traffic: It can be positively correlated with Bike demand. Higher traffic may force people to use bike as compared to other road transport medium like car, taxi etc.



<a id="2"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">2- Feature Engineering</p>

### New Features
- date (yyyy-mm-dd)
- month
- hour
- workingday
- holiday
- season
- battery_start
- battery_end
- path? (multi polygon)
- rental_duration


The number of rentals each hour will be aggregate later with a new feature `count`.

In [ ]:
rentals_data = historical_data.drop(['harvesttime','ebikestateid'], axis=1).copy()
rentals_data[["lastgpstime", "lastrentalstart"]] = rentals_data[["lastgpstime", "lastrentalstart"]].apply(pd.to_datetime)

rentals_data = rentals_data.astype({'battery': np.int16}, errors='ignore') # errors ignore to keep missing values (not throwing error)

In [ ]:
rentals_data.info()

### Rentals' information

- `coordinates`: converting latitude and longitude to an array to store a GeoJSON object *MultiPoint* 
- `start_battery`: getting the battery status when the rental started
- `lastgpstime`: new variable that will only store the last record when grouping rentals

In [ ]:
def feat_eng(x):
    d = {}
    d['coordinates'] = x[['latitude','longitude']].values.tolist()
    d['start_battery'] = list(x['battery'])[-1] # get the first battery status (when rental started)
    d['lastgpstime'] = list(x['lastgpstime'])[0] # get the last gpstime (previously sorted)
    
    return pd.Series(d, index=['coordinates', 'start_battery', 'lastgpstime'])

# also sorting data by lastgpstime
grouped_rentals = rentals_data.sort_values("lastgpstime", ascending=False).groupby(['lastrentalstart', 'bikeid']).apply(feat_eng).reset_index()

In [ ]:
grouped_rentals.shape

### Date and time - new features
- `rental_date`
- `rental_month`
- `rental_hour`
- `holiday`
- `workingday`
- `season`: (1 = Spring, 2 = Summer, 3 = Fall, 4 = Winter)
- *`duration`: duration of the rental

\* **Assumption**: that the duration of the rental is $ LastGPSTime - RentalStart $

In [ ]:
grouped_rentals['rental_date'] = pd.to_datetime(grouped_rentals['lastrentalstart'].dt.date)
grouped_rentals['rental_month'] = grouped_rentals['lastrentalstart'].dt.month
grouped_rentals['rental_hour'] = grouped_rentals['lastrentalstart'].dt.hour

In [ ]:
grouped_rentals.info()

In [ ]:
# time of rental in minutes (lastgpstime - rental-start)
grouped_rentals['duration'] = (grouped_rentals['lastgpstime'] - grouped_rentals['lastrentalstart']) / pd.Timedelta(minutes=1)

A few GPS records have frozen and stopped sending the accurate data back. About 345 records which would lead to a bias duration of rentals.

To prevent any inaccurate information these records will be set as `NaN`.

In [ ]:
grouped_rentals['duration'] = np.where(grouped_rentals['duration'] < 0, np.NaN, grouped_rentals['duration'])
len(grouped_rentals[ np.isnan(grouped_rentals['duration']) ])

## Bank Holidays

In [ ]:
qry_bh = {
    'type': 'National holiday'
}

bank_holidays = read_mongo(query=qry_bh, collection='irishcalendar')
bank_holidays.drop(['country', 'type'], axis=1, inplace=True)
bank_holidays['date'] = pd.DatetimeIndex(bank_holidays['date'].apply(pd.to_datetime))

In [ ]:
# holiday
grouped_rentals['holiday'] = grouped_rentals['rental_date'].isin(bank_holidays['date'])

# day of the week
grouped_rentals['dayofweek'] = grouped_rentals['rental_date'].dt.dayofweek

# working day (Monday=0, Sunday=6)
grouped_rentals['working_day'] = grouped_rentals['dayofweek'] < 5 # from 0 to 4 or monday to friday

## Seasons

In [ ]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [(3, (datetime(Y,  1,  1),  datetime(Y,  3, 20))),
           (0, (datetime(Y,  3, 21),  datetime(Y,  6, 20))),
           (1, (datetime(Y,  6, 21),  datetime(Y,  9, 22))),
           (2, (datetime(Y,  9, 23),  datetime(Y, 12, 20))),
           (3, (datetime(Y, 12, 21),  datetime(Y, 12, 31)))]

def get_season(date: pd.DatetimeIndex) -> int:
    '''
        Receives a date and returns the corresponded season
        0 - Spring | 1 - Summer | 2 - Autumn | 3 - Winter
        Vernal equinox(about March 21): day and night of equal length, marking the start of spring
        Summer solstice (June 20 or 21): longest day of the year, marking the start of summer
        Autumnal equinox(about September 23): day and night of equal length, marking the start of autumn
        Winter solstice (December 21 or 22): shortest day of the year, marking the start of winter
    '''
    date = date.replace(year=Y)
    return next(season for season, (start, end) in seasons if start <= date <= end)


grouped_rentals['season'] = grouped_rentals.rental_date.map(get_season)

In [ ]:
grouped_rentals.groupby('season').size()

In [ ]:
grouped_rentals.isnull().sum()

## Battery

In [ ]:
grouped_rentals['start_battery'] = pd.to_numeric(grouped_rentals['start_battery'])

In [ ]:
grouped_rentals[grouped_rentals['start_battery'] > 100]

From the battery records there is a few cases that we can consider. Only one record has `> 100` and a few negatives ones. To simplify the analysis the records will be normalized with values between `0 > x > 100`.

All missing values (*n=571*) will not be transformed as it could be only malfunction issue when transmiting the data and it could mislead the analysis.

In [ ]:
# normalize battery status between 0 > x < 100
grouped_rentals['start_battery'] = abs(grouped_rentals['start_battery'])
grouped_rentals.loc[grouped_rentals['start_battery'] > 100, 'start_battery'] = 100

In [ ]:
new_rentals = grouped_rentals.copy()
new_rentals.to_csv('../data/interim/new_features_rentals.csv')

## Humidity

Attempt to create a new feature called `Humidity` to avoid Multicollinearity.

### We need different humidity quantities

The problem with relative humidity is that, by itself, it doesn’t really tell you how humid it is.

- **Relative Humidity** – This quantity tells us how close the conditions are to saturation, when condensation of water vapor can occur. The interaction of porous materials with water vapor increases with increasing RH. The chance of growing mold increases with increasing RH, 70% usually given as the threshold to stay below.
- **Dew Point Temperature** – This temperature scales with the amount of water vapor. As more water vapor enters a volume, the dew point goes up. If the air in your crawl space, for example, has a dew point of 75° F, you’re probably going to find condensation somewhere. Look at the water pipes, poorly insulated ducts, and uninsulated duct boots.
- **Wet Bulb Temperature** – If dew point is the temperature of condensation, wet bulb is the temperature of evaporation. Same concept; different direction. This one’s important for cooling our bodies.

Once you get a handle on these three quantities, you’ll have a pretty good understanding of humidity.

> https://www.energyvanguard.com/blog/problem-with-relative-humidity

In [ ]:
weather_data.head()

In [ ]:
weather_data['rhum'] = pd.to_numeric(weather_data['rhum'],errors = 'coerce')
weather_data['wetb'] = pd.to_numeric(weather_data['wetb'],errors = 'coerce')
weather_data['dewpt'] = pd.to_numeric(weather_data['dewpt'],errors = 'coerce')

### Variance Inflation Factor(VIF)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
test_vif = weather_data.copy().drop('date', axis=1)
#Calculate VIF for each variable in the new data frame
vif = pd.DataFrame()
vif["features"] = test_vif.columns
vif["vif_value"] = [variance_inflation_factor(test_vif.values, i) for i in range(test_vif.shape[1])]
vif

To avoid multicolinearity we could create a new feature based on some calculation on those three features related to humidity. Unfortunately, the Forecast API later that will be used in production to predict the demand **does not include Wet Bulb Temperature**.

There are a few equations that could applied to calculate Wet Bulb Temperature (eg: Stull formula) but because we do not have which one is used to store this feature on their historical the best would be not use this feature at all.

In [ ]:
#weather_data.drop(['rhum', 'wetb', 'dewpt'], axis=1, inplace=True)

weather_data.drop(['wetb', 'dewpt'], axis=1, inplace=True)

## Combining Rentals and Weather data

In [ ]:
rentals = new_rentals.copy()
weather = weather_data.copy()

weather['rental_date'] = pd.to_datetime(weather['date'].dt.date)
weather['rental_hour'] = weather['date'].dt.hour

In [ ]:
rentals.groupby('season').size()

In [ ]:
all_data = pd.merge(rentals, weather, on=['rental_date', 'rental_hour'])
all_data.to_csv('../data/interim/all_data.csv')

In [ ]:
all_data.groupby('season').size()

<a id="3"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">3- Exploratory Data Analysis (EDA)</p>

- Hourly trend
- Holiday
- Working_day
- Season

In [ ]:
hourly_rentals = all_data.copy()
count_hourly_rentals = hourly_rentals.groupby(['rental_date', 'rental_hour']).size().reset_index(name='count')
columns_to_drop = ['lastrentalstart','bikeid','coordinates','start_battery','lastgpstime','rental_month','duration','date']
hourly_rentals = hourly_rentals.drop(columns_to_drop, axis=1).drop_duplicates(subset=['rental_date', 'rental_hour'])

In [ ]:
hourly_rentals.shape, count_hourly_rentals.shape

In [ ]:
hourly_data = pd.merge(hourly_rentals, count_hourly_rentals, on=['rental_date','rental_hour'])
hourly_data.to_csv('../data/interim/hourly_data.csv')

In [ ]:
hourly_data.head()

## Viz

- Hour x Count by Season
- Hour x Count by day of week
- Hour x Count by temperature
- Battery
- Duration of rentals

In [ ]:
season_freq = hourly_data.groupby(['season'])['count'].agg('sum').reset_index(name='count')
season_freq

In [ ]:
# 0 - Spring | 1 - Summer | 2 - Autumn | 3 - Winter
season_map = {0:'Spring', 1:'Summer', 2:'Fall', 3:'Winter'}
season_freq = hourly_data.groupby(['season', 'rental_hour'])['count'].agg('sum').reset_index(name='count')
season_freq['season'] = season_freq['season'].map(lambda d : season_map[d])
fig, ax = plt.subplots(figsize=(18, 10))
sns.pointplot(x=season_freq["rental_hour"], y=season_freq["count"], hue=season_freq["season"], ax=ax)
ax.set(xlabel='Hour Of The Day', ylabel='Rentals Count', title="Number of Rentals By Hour Of The Day Across Seasons")
plt.show()

### Note

Summer data is very low due to the last data collect is June 30th and the summer have just started. **Decided to not use the season as a predictor** for now and after the changing of season the model will be rebuild adding this feature.

In [ ]:
day_of_week_map = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
day_of_week = hourly_data.groupby(['dayofweek', 'rental_hour'])['count'].agg('sum').reset_index(name='count')
day_of_week['dayofweek'] = day_of_week['dayofweek'].map(lambda d : day_of_week_map[d])
fig, ax = plt.subplots(figsize=(18, 10))
sns.pointplot(x=day_of_week["rental_hour"], y=day_of_week["count"], hue=day_of_week["dayofweek"], ax=ax)
ax.set(xlabel='Hour Of The Day', ylabel='Rentals Count', title="Number of Rentals By Hour Of The Day Across Days of Week")
plt.show()

In [ ]:
battery_dist = all_data.copy()

def group_battery_status(df):

    bins= [0,30,50,80,100]
    labels = ['< 30%','30% - 50%','50% - 80%','> 80%']
    battery_dist['battery_status'] = pd.cut(battery_dist['start_battery'], bins=bins, labels=labels, right=False)

    s = battery_dist.battery_status
    counts = s.value_counts()
    percent = s.value_counts(normalize=True)
    percent100 = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
    
    return pd.DataFrame({'counts': counts, 'per': percent, 'per100': percent100}, labels)

In [ ]:
group_battery_status(battery_dist)

In [ ]:
sns.histplot(data=new_rentals, x='start_battery', kde=True)

In [ ]:
all_data['duration'].mean()

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sns.boxplot(data=hourly_data,y="count",orient="v",ax=axes[0][0])
sns.boxplot(data=hourly_data,y="count",x="season",orient="v",ax=axes[0][1])
sns.boxplot(data=hourly_data,y="count",x="rental_hour",orient="v",ax=axes[1][0])
sns.boxplot(data=hourly_data,y="count",x="working_day",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Box Plot On Count Across Season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")

plt.show()

In [ ]:
corrMatt = hourly_data[["temp","rain","wdsp",'rhum',"count"]].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(8,8)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.scatter(hourly_data['count'], hourly_data['temp'])

plt.show()

In [ ]:
fig = plt.figure(figsize=(22,7))
gs = fig.add_gridspec(1, 3)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])

sns.distplot(hourly_data['temp'],ax=ax0)
sns.distplot(hourly_data['rain'],ax=ax1)
sns.distplot(hourly_data['wdsp'],ax=ax2)

ax0.set(xlabel='Temperature',ylabel='Count',title="Distribution - Temperature")
ax1.set(xlabel='Rain', ylabel='Count',title="Distribution - Rain")
ax2.set(xlabel='Wind Speed', ylabel='Count',title="Distribution - Wind Speed")

plt.show()

<a id="4"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">4- Models</p>

## Splitting dataset in train and test

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
df = hourly_data.copy()
df = df.astype({'holiday': 'category', 'dayofweek': 'category', 'working_day': 'category'})
predictors = ['rain','temp','wdsp','rhum','holiday','dayofweek','working_day']

X = df[[c for c in df.columns if c in predictors]]
y = df.pop('count')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.head()

## Split columns in categorical and numerical to use preprocessing pipeline

In [ ]:
num_vars = [c for c in df.select_dtypes(include=['number']).columns if c in predictors] # list comprehension to select only predictors features
cat_vars = df.select_dtypes(include=['category']).columns

## Preprocessing Pipelines

In [ ]:
# Define categorical pipeline
cat_pipe = Pipeline([
    #('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

# Define numerical pipeline
num_pipe = Pipeline([
    #('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Combine categorical and numerical pipelines
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, cat_vars),
    ('num', num_pipe, num_vars)
])

In [ ]:
# Fit a pipeline with transformers and an estimator to the training data
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
pipe.fit(X_train, y_train)

In [ ]:
# Predict training data
y_train_pred = pipe.predict(X_train)
print(f"Predictions on training data: {y_train_pred}")

# Predict test data
y_test_pred = pipe.predict(X_test)
print(f"Predictions on test data: {y_test_pred}")

In [ ]:
def adjusted_r2_score(X, r2_score):
    return 1 - ( 1- r2_score ) * ( len(X) - 1 ) / ( len(X) - X.shape[1] - 1 )

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred)) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
print('R^2:', metrics.r2_score(y_test, y_test_pred))
print('Adjusted R^2:', adjusted_r2_score(X_test, metrics.r2_score(y_test, y_test_pred)))

### Ridge
[Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of the coefficients with l2 regularization.

### Lasso
The [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso) is a linear model that estimates sparse coefficients with l1 regularization.

### ElasticNet
[Elastic-Net](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) is a linear regression model trained with both l1 and l2 -norm regularization of the coefficients.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

## Ridge Regression

In [ ]:
# Fit a pipeline with transformers and an estimator to the training data
pipe_ridge = Pipeline([
    ('preprocessor', preprocessor),
    ('model', Ridge())
])
pipe_ridge.fit(X_train, y_train)

In [ ]:
y_train_pred = pipe_ridge.predict(X_train)
y_test_pred = pipe_ridge.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred)) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
print('R^2:', metrics.r2_score(y_test, y_test_pred))
print('Adjusted R^2:', adjusted_r2_score(X_test, metrics.r2_score(y_test, y_test_pred)))

## Lasso Regression

In [ ]:
# Fit a pipeline with transformers and an estimator to the training data
pipe_lasso = Pipeline([
    ('preprocessor', preprocessor),
    ('model', linear_model.Lasso(alpha=0.1))
])
pipe_lasso.fit(X_train, y_train)
y_train_pred = pipe_lasso.predict(X_train)
y_test_pred = pipe_lasso.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred)) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
print('R^2:', metrics.r2_score(y_test, y_test_pred))
print('Adjusted R^2:', adjusted_r2_score(X_test, metrics.r2_score(y_test, y_test_pred)))

## Elastic-Net

In [ ]:
# Fit a pipeline with transformers and an estimator to the training data
pipe_elastic = Pipeline([
    ('preprocessor', preprocessor),
    ('model', ElasticNet())
])
pipe_elastic.fit(X_train, y_train)
y_train_pred = pipe_elastic.predict(X_train)
y_test_pred = pipe_elastic.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred)) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
print('R^2:', metrics.r2_score(y_test, y_test_pred))
print('Adjusted R^2:', adjusted_r2_score(X_test, metrics.r2_score(y_test, y_test_pred)))

<img title="GitHub Mark" src="./img/GitHub-Mark-64px.png" style="height: 32px; padding-right: 15px" alt="GitHub Mark" align="left"> [GitHub repository](https://github.com/pessini/moby-bikes) <br>Author: Leandro Pessini